### 1. Preparação dos Dados

In [11]:
# Importar bibliotecas
import os
import keras
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras.layers import Dense
from tensorflow.keras.preprocessing import image
import numpy as np

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
# Definir caminho do dataset
root = 'drive/MyDrive/DIO/Projetos/01-Transfer Learning/101_ObjectCategories/'
exclude = ['BACKGROUND_Google', 'Motorbikes', 'airplanes', 'Faces_easy', 'Faces']

# Definir splits dos dados
train_split, val_split = 0.7, 0.15  # 70% treino, 15% validação, 15% teste

In [18]:
# Preprocessamento das imagens
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

categories = [x[0] for x in os.walk(root) if x[0]][1:]
categories = [c for c in categories if c not in [os.path.join(root, e) for e in exclude]]
num_classes = len(categories)
print(num_classes)

97


### 2. Carregar o Modelo Pré-treinado (VGG16)

In [19]:
# Carregar VGG16 pré-treinado no ImageNet
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)

### 3. Modificar o Modelo para Transfer Learning

In [20]:
# Fazer referência à camada de entrada do VGG
inp = vgg.input

# Criar nova camada de classificação para suas classes
new_classification_layer = Dense(num_classes, activation='softmax')

# Conectar nova camada à penúltima camada do VGG
out = new_classification_layer(vgg.layers[-2].output)

# Criar novo modelo
model_new = Model(inp, out)

### 4. Congelar as Camadas do VGG16

In [21]:
# Congelar todas as camadas exceto a última
for layer in model_new.layers[:-1]:
    layer.trainable = False

# Garantir que a última camada está treinável
model_new.layers[-1].trainable = True

### 5. Compilar e Treinar o Modelo

In [22]:
# Compilar o modelo
model_new.compile(loss='categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])

# Treinar o modelo
history = model_new.fit(x_train, y_train,
                       batch_size=128,
                       epochs=10,
                       validation_data=(x_val, y_val))

NameError: name 'x_train' is not defined

### 6. Avaliar o Modelo

In [ ]:
# Avaliar no conjunto de teste
loss, accuracy = model_new.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', accuracy)

### 7. Fazer Predições


In [ ]:
# Predizer uma nova imagem
img, x = get_image('caminho_da_imagem.jpg')
probabilities = model_new.predict([x])